In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
import rich

In [2]:
load_dotenv()

True

In [4]:
OPENROUTER_API_KEY  = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL")
if not OPENROUTER_API_KEY:
    raise RuntimeError("Missing OPENROUTER_API_KEY in .env")
if not OPENROUTER_BASE_URL:
    raise RuntimeError("Missing OPENROUTER_BASE_URL in .env")

In [6]:
client = OpenAI(api_key=OPENROUTER_API_KEY, base_url=OPENROUTER_BASE_URL)

In [14]:
# Listing 3.1
calculator_tool_definition = {
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Perform basic arithmetic operations between two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "operator": {
                    "type": "string",
                    "description": "Arithmetic operation to perform",
                    "enum": ["add", "subtract", "multiply", "divide"]
                },
                "first_number": {
                    "type": "number",
                    "description": "First number for the calculation"
                },
                "second_number": {
                    "type": "number",
                    "description": "Second number for the calculation"
                }
            },
            "required": ["operator", "first_number", "second_number"],
        }
    }
}

In [9]:
# Listing 3.2
def calculator(operator: str, first_number: float, second_number: float) -> float:
   if operator == 'add':
       return first_number + second_number
   elif operator == 'subtract':
       return first_number - second_number
   elif operator == 'multiply':
       return first_number * second_number
   elif operator == 'divide':
       if second_number == 0:
           raise ValueError("Cannot divide by zero")
       return first_number / second_number
   else:
       raise ValueError(f"Unsupported operator: {operator}")

In [15]:
# Listing 3.3
tools = [calculator_tool_definition]

response_without_tool = client.chat.completions.create(
        model='openai/gpt-4o-mini',
        messages=[{"role": "user", "content": "What is the capital of South Korea?"}],
        tools=tools
)

In [16]:
rich.print(response_without_tool)

ChatCompletion(
    id='gen-1763010328-PldfhFSgNaXy10rZqNbC',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='The capital of South Korea is Seoul.',
                refusal=None,
                role='assistant',
                annotations=None,
                audio=None,
                function_call=None,
                tool_calls=None,
                reasoning=None
            ),
            native_finish_reason='stop'
        )
    ],
    created=1763010328,
    model='openai/gpt-4o-mini',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_560af6e559',
    usage=CompletionUsage(
        completion_tokens=9,
        prompt_tokens=93,
        total_tokens=102,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=None,
            audio_tokens=None,
            reasoning_tokens=0,
            rejected_prediction_tokens=None
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    ),
    provider='OpenAI'
)

In [17]:
# Plain print for easy copying
print(response_without_tool)

ChatCompletion(id='gen-1763010328-PldfhFSgNaXy10rZqNbC', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of South Korea is Seoul.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1763010328, model='openai/gpt-4o-mini', object='chat.completion', service_tier=None, system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=9, prompt_tokens=93, total_tokens=102, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), provider='OpenAI')


In [18]:
print(response_without_tool.choices[0].message.content)

The capital of South Korea is Seoul.


In [19]:
print(response_without_tool.choices[0].message.tool_calls)

None


In [20]:
response_with_tool = client.chat.completions.create(
        model='openai/gpt-4o-mini',
        messages=[{"role": "user", "content": "What is 1234 x 5678?"}],
        tools=tools
)

In [21]:
print(response_with_tool)

ChatCompletion(id='gen-1763010337-QR87xVtBJgcM2aPLH3E7', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_IShCxALT7DpZ78ufDs2SoJvw', function=Function(arguments='{"operator":"multiply","first_number":1234,"second_number":5678}', name='calculator'), type='function', index=0)], reasoning=None), native_finish_reason='tool_calls')], created=1763010337, model='openai/gpt-4o-mini', object='chat.completion', service_tier=None, system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=26, prompt_tokens=95, total_tokens=121, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), provider='OpenAI')


In [22]:
print(response_with_tool.choices[0].message.content)

In [23]:
print(response_with_tool.choices[0].message.tool_calls)

[ChatCompletionMessageFunctionToolCall(id='call_IShCxALT7DpZ78ufDs2SoJvw', function=Function(arguments='{"operator":"multiply","first_number":1234,"second_number":5678}', name='calculator'), type='function', index=0)]


In [24]:
# Listing 3.4
ai_message = response_with_tool.choices[0].message

if ai_message.tool_calls:
   for tool_call in ai_message.tool_calls:
       function_name = tool_call.function.name
       function_args = json.loads(tool_call.function.arguments)

       if function_name == "calculator":
           result = calculator(**function_args)

In [25]:
# Listing 3.5
messages = []
messages.append({"role": "user", "content": "What is 1234 x 5678?"})

response_with_tool = client.chat.completions.create(
    model='openai/gpt-4o-mini',
    messages=messages,
    tools=tools
)

ai_message = response_with_tool.choices[0].message

messages.append({
    "role": "assistant",
    "content": ai_message.content,
    "tool_calls": ai_message.tool_calls
})


In [26]:
if ai_message.tool_calls:
        for tool_call in ai_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            if function_name == "calculator":
                result = calculator(**function_args)

                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })

In [27]:
    final_response = client.chat.completions.create(
        model='gpt-5-mini',
        messages=messages
    )

In [28]:
    print("Messages:", messages)
    print("Final Answer:", final_response.choices[0].message.content)

Messages: [{'role': 'user', 'content': 'What is 1234 x 5678?'}, {'role': 'assistant', 'content': '', 'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_oTVuhmMdqR7F7no2FKOeCYys', function=Function(arguments='{"operator":"multiply","first_number":1234,"second_number":5678}', name='calculator'), type='function', index=0)]}, {'role': 'tool', 'tool_call_id': 'call_oTVuhmMdqR7F7no2FKOeCYys', 'content': '7006652'}]
Final Answer: 1234 × 5678 = 7,006,652.

Quick check: 1234×(5000+600+70+8) = 6,170,000 + 740,400 + 86,380 + 9,872 = 7,006,652.
